In [1]:
from vtk import *

In [2]:
reader = vtkXMLImageDataReader()
reader.SetFileName('Data/Isabel_2D.vti')
reader.Update()
data = reader.GetOutput()

# Question 1

### 1.1 Number of cells in the Dataset

In [3]:
numCells = data.GetNumberOfCells()

In [4]:
print("Number of cells in the dataset:",numCells)

Number of cells in the dataset: 62001


### 1.2 The dimensions of the dataset

In [5]:
Dimension=data.GetDimensions()

In [6]:
print(" Dimensions of the dataset:",Dimension)

 Dimensions of the dataset: (250, 250, 1)


### 1.3 The number of points present in the uniform grid of the data

In [7]:
Number_of_points=Dimension[0]*Dimension[1]*Dimension[2]

In [8]:
print("The number of points present in the uniform grid of the data:",Number_of_points)

The number of points present in the uniform grid of the data: 62500


### 1.4 Print the range of Pressure values present in the dataset

In [9]:
surface=vtkGeometryFilter()
surface.SetInputData(data)
surface.Update()
Pressure_Data=surface.GetOutput()
Range=Pressure_Data.GetPointData().GetArray('Pressure').GetRange()

In [10]:
print("The range of Pressure values present in the dataset:",Range)

The range of Pressure values present in the dataset: (-1434.8590087890625, 630.5694580078125)


### 1.5 Print the average Pressure value of the entire dataset

In [11]:
dataArr = data.GetPointData().GetArray('Pressure')

In [12]:
sum_pre=0
for i in range(dataArr.GetNumberOfTuples()):
     sum_pre+=dataArr.GetTuple1(i)
avg_pressure=sum_pre/dataArr.GetNumberOfTuples()
print("Print the average Pressure value of the entire dataset",avg_pressure)


Print the average Pressure value of the entire dataset 240.77722069091325


### 1.6 Extract a vtkCell object with cell id=0, i.e., the first cell

In [13]:
### Get a single cell from the list of cells
cell_id=0
cell = data.GetCell(cell_id)

### 1.7 Print the indices of the four corner vertices of the cell

In [14]:
#  4 corner points of the cell with cell id=0

pid1 = cell.GetPointId(0)
pid2 = cell.GetPointId(1)
pid3 = cell.GetPointId(3)
pid4 = cell.GetPointId(2)

print('The indices of the four corner vertices of the cell:')
print(pid1,pid2,pid3,pid4) ## in counter-clockwise order

The indices of the four corner vertices of the cell:
0 1 251 250


### 1.8 Print the 3D coordinate of each vertex

In [15]:
## Print the locations (3D coordinates) of the points
print('3D coordinate of each vertex:') #in counter clockwise order
print(data.GetPoint(pid1))
print(data.GetPoint(pid2))
print(data.GetPoint(pid3))
print(data.GetPoint(pid4))

3D coordinate of each vertex:
(0.0, 0.0, 25.0)
(1.0, 0.0, 25.0)
(1.0, 1.0, 25.0)
(0.0, 1.0, 25.0)


### 1.9 Compute the 3D coordinate of the cell center using its four corner vertices

In [16]:
avg_x=(data.GetPoint(pid1)[0]+data.GetPoint(pid2)[0]+data.GetPoint(pid3)[0]+data.GetPoint(pid4)[0])/4
avg_y=(data.GetPoint(pid1)[1]+data.GetPoint(pid2)[1]+data.GetPoint(pid3)[1]+data.GetPoint(pid4)[1])/4
avg_z=(data.GetPoint(pid1)[2]+data.GetPoint(pid2)[2]+data.GetPoint(pid3)[2]+data.GetPoint(pid4)[2])/4
center_coordinate=(avg_x,avg_y,avg_z)
print("3D coordinate of the cell center using its four corner vertices",center_coordinate)

3D coordinate of the cell center using its four corner vertices (0.5, 0.5, 25.0)


### 1.10 Print the data value (Pressure) for all the four vertices of the extracted cell

In [17]:
dataArr = data.GetPointData().GetArray('Pressure')
val1 = dataArr.GetTuple1(pid1)
val2 = dataArr.GetTuple1(pid2)
val3 = dataArr.GetTuple1(pid3)
val4 = dataArr.GetTuple1(pid4)
print("Pressure for all the four vertices of the extracted cell.:",val1,val2,val3,val4)

Pressure for all the four vertices of the extracted cell.: 477.527587890625 474.79827880859375 467.60699462890625 478.0115661621094


### 1.11 Print the mean (average) Pressure value at the cell center

In [18]:
avg_pressure_extracted_cell=(val1+val2+val3+val4)/4
print("Mean pressure of extracted cell by averaging pressure:",avg_pressure_extracted_cell)

Mean pressure of extracted cell by averaging pressure: 474.4861068725586


# Question 2

In [19]:
points = vtkPoints()
points.InsertNextPoint(data.GetPoint(pid1))
points.InsertNextPoint(data.GetPoint(pid2))
points.InsertNextPoint(data.GetPoint(pid3))
points.InsertNextPoint(data.GetPoint(pid4))

### Create a polydata
polydata= vtkPolyData()

### Add points to polydata
polydata.SetPoints(points)

In [20]:
# Create an array of colors
colors = vtkUnsignedCharArray()
colors.SetNumberOfComponents(3)
colors.SetName("Colors")
colors.InsertNextTuple3(255, 0, 0) # red
colors.InsertNextTuple3(0, 255, 0) # green
colors.InsertNextTuple3(0, 0, 255) # blue
colors.InsertNextTuple3(21, 244, 238) # fluorescent blue
polydata.GetPointData().SetScalars(colors)

# Create the vertex glyph filter
glyphFilter = vtkVertexGlyphFilter()
glyphFilter.SetInputData(polydata)
glyphFilter.Update()

In [21]:
# Create the mapper, actor, and renderer
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(glyphFilter.GetOutputPort())
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetPointSize(30)
renderer = vtkRenderer()
renderer.AddActor(actor)

In [22]:
# Display the renderer
renWin = vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
iren.Initialize()
iren.Start()